## Youtube Data API

`pip install google-auth-oauthlib`   
`pip install google-api-python-client`

In [1]:
import os
import io
import googleapiclient.discovery
import googleapiclient.errors

from googleapiclient.http import MediaIoBaseDownload

import pprint
pp = pprint.PrettyPrinter(indent=4)

scopes = ["https://www.googleapis.com/auth/youtube.force-ssl" ]
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
api_service_name = "youtube"
api_version  = "v3"

###### OAuth 2.0   

In [12]:
from google.oauth2 import service_account

SERVICE_ACCOUNT_FILE = 'ml-service.json'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=scopes)

youtube = googleapiclient.discovery.build(
    api_service_name, 
    api_version, 
    credentials=credentials)

### Video Captions
https://developers.google.com/youtube/v3/docs/captions
   
Associated with one YouTube video. 

##### API Methods   
`download` - GET / captions / <em> id </em> : Downloads a caption track.   
`list` - GET / captions : Returns list of caption tracks that are associated with a specified video.


###### Captions `list`   
Quota cost of 50 units.

In [5]:
request = youtube.captions().list(
        videoId="ukTaodQfYRQ",
        part="snippet"
)
response = request.execute()
#pp.pprint(response)
engCapResource = englishCaptionResource(response)
pp.pprint(engCapResource)
engCaptionId = engCapResource['id']

{   'etag': 'tNIpRCq_st0FcYjK7dLbY9QKpjU',
    'id': '_Z1lGXQWnk6cX8g4GP7lKnxpXAT8rVSSP0_hnIQbviI=',
    'kind': 'youtube#caption',
    'snippet': {   'audioTrackType': 'unknown',
                   'isAutoSynced': False,
                   'isCC': False,
                   'isDraft': False,
                   'isEasyReader': False,
                   'isLarge': False,
                   'language': 'en',
                   'lastUpdated': '2020-09-03T03:00:32.776000Z',
                   'name': '',
                   'status': 'serving',
                   'trackKind': 'asr',
                   'videoId': 'ukTaodQfYRQ'}}


In [4]:
# takes response of GET/captions method and extracts english caption resource.
def englishCaptionResource(res):
    items = response['items']
    for item in items:
        if (item['snippet']['language'] == 'en'):
            return item

##### Captions `download`
Quota cost of 200 units.   
Only works for videos that have enabled third-party contributions.

In [6]:
request = youtube.captions().download(
        id=engCaptionId
    )

fh = io.FileIO("caption_data.txt", "wb")
download = MediaIoBaseDownload(fh, request)
complete = False
while not complete:
    status, complete = download.next_chunk()

### CommentThreads
https://developers.google.com/youtube/v3/docs/commentThreads   
   
`commentThread` resource can represent comments about either a video or a channel.   
   
##### API Methods    
`list` - GET / captions : Returns list of comment threads that match the API request parameters.

In [13]:
request = youtube.commentThreads().list(
        part="snippet,replies",
        videoId="ukTaodQfYRQ"
    )
response = request.execute()
pp.pprint(response)

{   'etag': '9cdkunknZs5xdTXtEdPLMNsrLMs',
    'items': [   {   'etag': 'oPK2mLD5zeurcXW8MKTTDL1b4_0',
                     'id': 'UgycBMtpJUMsQ-KIcdF4AaABAg',
                     'kind': 'youtube#commentThread',
                     'snippet': {   'canReply': True,
                                    'isPublic': True,
                                    'topLevelComment': {   'etag': 'bTDleFRF3PrfajZNdV4uxjBIy8g',
                                                           'id': 'UgycBMtpJUMsQ-KIcdF4AaABAg',
                                                           'kind': 'youtube#comment',
                                                           'snippet': {   'authorChannelId': {   'value': 'UCTaSaFOQ_RFeyHW1g__3log'},
                                                                          'authorChannelUrl': 'http://www.youtube.com/channel/UCTaSaFOQ_RFeyHW1g__3log',
                                                                          'authorDisplayName': 'The '
         

## Cloud Natural Language API

Python Library   
https://googleapis.dev/python/language/latest/usage.html   
   
`pip install google-cloud-language`
  
https://cloud.google.com/natural-language/docs/reference/libraries#windows

In [10]:
#from google.oauth2 import service_account
#api_service_name = "youtube"
#api_version  = "v1"
import os

from google.cloud import language_v1
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:\\Users\\Ernest\\nlp-key.json'
client = language_v1.LanguageServiceClient()

text_content = 'panic attack so I was looking for'

# Available types: PLAIN_TEXT, HTML
type_ = language_v1.Document.Type.PLAIN_TEXT

# Optional. If not specified, the language is automatically detected.
# For list of supported languages:
# https://cloud.google.com/natural-language/docs/languages
language = "en"

document = language_v1.Document(content=text_content, type_=type_)
# Available values: NONE, UTF8, UTF16, UTF32
encoding_type = language_v1.EncodingType.UTF8

response = client.analyze_entities(request = {'document': document, 'encoding_type': encoding_type})
print(response)

entities {
  name: "panic attack"
  type_: EVENT
  salience: 1.0
  mentions {
    text {
      content: "panic attack"
    }
    type_: COMMON
  }
}
language: "en"

